# Tourist satisfaction with transport systems to Mount Etna, Sicily. Part 3. Predict.

# This cell will prompt you to connect this notebook with your google account.
from google.colab import drive                                                                        
drive.mount('/content/gdrive', force_remount=True)
root_dir = "/content/gdrive/My Drive/"
base_dir = root_dir + 'Academy/+ Papers/en_proceso/mount_etna/'


In [1]:
base_dir = '../' # To run locally

# 1. Data pre-processing

In [2]:
#!pip install pycaret[full]

## Importing libraries

In [3]:
import pandas as pd
import numpy as np
import re
import matplotlib.pyplot as plt
import seaborn as sns
from wordcloud import WordCloud
from sklearn.model_selection import RepeatedStratifiedKFold, cross_val_score
import imblearn
from pycaret.classification import *
import spacy
#spacy.load("en_core_web_sm")


# NLTK
import string
import nltk
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('punkt')
nltk.download('words')
nltk.download('averaged_perceptron_tagger')
from nltk.corpus import wordnet, stopwords
from collections import Counter
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import CountVectorizer

#ignore log(0) and divide by 0 warning
np.seterr(divide = 'ignore');
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings("ignore", category=DeprecationWarning)

print('Libraries read!')

Libraries read!


[nltk_data] Downloading package stopwords to /home/juan/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /home/juan/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package punkt to /home/juan/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package words to /home/juan/nltk_data...
[nltk_data]   Package words is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /home/juan/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


# Reading models

In [4]:
model_title = load_model(base_dir + 'model/model_title')
model_review = load_model(base_dir + 'model/model_review')
model_classification = load_model(base_dir + 'model/classification')

Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded
Transformation Pipeline and Model Successfully Loaded


## Reading data

In [5]:
data = pd.read_csv(base_dir + 'predict/data_to_predict.csv')

# Dropping rows without data in content
data = data.dropna(subset = ['content'])

print('Data shape: ', data.shape)
data.head()

Data shape:  (17, 2)


,content,title
0,"East side of the sicilian island, in the Catan...",Etna🇮🇹
1,August 2008. Arrived at Refugio Sapienza just ...,"A hard climb, but worth it"
2,Mount Etna is extraordinary. It is an active v...,You must see an active volcano at least once i...
3,How much does the cable car cost from Rifugio ...,Cost of cable car to the top???
4,It's always a great and sensational emotion to...,Wonderful Etna


## Filtering comments according to keywords

In [6]:
TRANSPORT_KEYWORDS = 'transport|rail|cable|car|bus|transportation|mobility|bike'

In [7]:
# Filtering
data['is_transport_related'] = data['content'].str.contains(TRANSPORT_KEYWORDS, case=False, na=False)
data = data[data['is_transport_related']==True].reset_index(drop=True)
data = data.drop(columns=['is_transport_related'])

# New dataset
print('Data shape: ', data.shape)
data.head(10)

Data shape:  (10, 2)


,content,title
0,August 2008. Arrived at Refugio Sapienza just ...,"A hard climb, but worth it"
1,Mount Etna is extraordinary. It is an active v...,You must see an active volcano at least once i...
2,How much does the cable car cost from Rifugio ...,Cost of cable car to the top???
3,We visited Etna with two small children in Aug...,"Well orgainised, easy access"
4,We were able to experience Mt Etna without spe...,Cheap and easy way to do Mt Etna
5,We would like to visit Etna. We have a 3 year ...,Question about Etna Cable Cars and children
6,Mount Etna is a stunning day out from Catania....,"Mount Etna - AST bus from Catania, cable car a..."
7,"If you are in good condition, you can climb th...",walk mount etna on your own
8,We stayed at Nicolosi 25 k's from Mt Etna. It ...,Stunning
9,"Mt Etna is of course a ""must visit"", but be wa...",Watch out for the clouds...


In [8]:
noise_words = []
stopwords_corpus = nltk.corpus.stopwords
eng_stop_words = stopwords_corpus.words('english')
noise_words.extend(eng_stop_words)

In [9]:
data

,content,title
0,August 2008. Arrived at Refugio Sapienza just ...,"A hard climb, but worth it"
1,Mount Etna is extraordinary. It is an active v...,You must see an active volcano at least once i...
2,How much does the cable car cost from Rifugio ...,Cost of cable car to the top???
3,We visited Etna with two small children in Aug...,"Well orgainised, easy access"
4,We were able to experience Mt Etna without spe...,Cheap and easy way to do Mt Etna
5,We would like to visit Etna. We have a 3 year ...,Question about Etna Cable Cars and children
6,Mount Etna is a stunning day out from Catania....,"Mount Etna - AST bus from Catania, cable car a..."
7,"If you are in good condition, you can climb th...",walk mount etna on your own
8,We stayed at Nicolosi 25 k's from Mt Etna. It ...,Stunning
9,"Mt Etna is of course a ""must visit"", but be wa...",Watch out for the clouds...


# 3. NLP for title

In [10]:
# Pycaret
from pycaret.nlp import *

exp_name = setup(data = data[['title']],  
                 target = 'title',
                 session_id = 42,
                 custom_stopwords = noise_words
                )

Description,Value
session_id,42
Documents,10
Vocab Size,20
Custom Stopwords,True


In [11]:
print(model_title)

LdaModel(num_terms=795, num_topics=3, decay=0.5, chunksize=100)


In [12]:
predictions_title = assign_model(model_title)  
predictions_title

,title,Topic_0,Topic_1,Topic_2,Dominant_Topic,Perc_Dominant_Topic
0,hard,0.166956,0.666083,0.166961,Topic 1,0.67
1,must see active volcano least life,0.903465,0.048775,0.047760,Topic 0,0.90
2,cost cable car top,0.066697,0.066736,0.866568,Topic 2,0.87
3,orgainise easy access,0.086296,0.083887,0.829817,Topic 2,0.83
4,cheap easy way,0.809144,0.083460,0.107396,Topic 0,0.81
5,question cable car child,0.069453,0.067109,0.863438,Topic 2,0.86
6,cable car,0.111141,0.111143,0.777716,Topic 2,0.78
7,,0.333333,0.333333,0.333333,Topic 0,0.33
8,,0.333333,0.333333,0.333333,Topic 0,0.33
9,watch cloud,0.111266,0.774443,0.114291,Topic 1,0.77


In [13]:
predictions_title = predictions_title.add_prefix('Title_')
predictions_title['Title_Dominant_Topic'] = predictions_title['Title_Dominant_Topic'].replace(' ', '_', regex=True)
predictions_title

,Title_title,Title_Topic_0,Title_Topic_1,Title_Topic_2,Title_Dominant_Topic,Title_Perc_Dominant_Topic
0,hard,0.166956,0.666083,0.166961,Topic_1,0.67
1,must see active volcano least life,0.903465,0.048775,0.047760,Topic_0,0.90
2,cost cable car top,0.066697,0.066736,0.866568,Topic_2,0.87
3,orgainise easy access,0.086296,0.083887,0.829817,Topic_2,0.83
4,cheap easy way,0.809144,0.083460,0.107396,Topic_0,0.81
5,question cable car child,0.069453,0.067109,0.863438,Topic_2,0.86
6,cable car,0.111141,0.111143,0.777716,Topic_2,0.78
7,,0.333333,0.333333,0.333333,Topic_0,0.33
8,,0.333333,0.333333,0.333333,Topic_0,0.33
9,watch cloud,0.111266,0.774443,0.114291,Topic_1,0.77


# 4. NLP for review

In [14]:
exp_name = setup(data = data[['content']], 
                     target = 'content',
                     session_id = 42,
                     custom_stopwords = noise_words
                     )

Description,Value
session_id,42
Documents,10
Vocab Size,433
Custom Stopwords,True


In [15]:
print(model_review)

LdaModel(num_terms=5943, num_topics=2, decay=0.5, chunksize=100)


In [16]:
predictions_review = assign_model(model_review)  
predictions_review

,content,Topic_0,Topic_1,Dominant_Topic,Perc_Dominant_Topic
0,arrive make sure allow plenty time mountain ro...,0.629743,0.370257,Topic 0,0.63
1,extraordinary active volcano beautiful landsca...,0.773659,0.226341,Topic 0,0.77
2,much cable car cost charge really look receipt...,0.479131,0.520869,Topic 1,0.52
3,small child august use company get see sight h...,0.202369,0.797631,Topic 1,0.80
4,able experience etna spend money tour research...,0.352371,0.647629,Topic 1,0.65
5,would visit old month old know take child youn...,0.443988,0.556012,Topic 1,0.56
6,take follow cable car star deduction bus trip ...,0.417466,0.582534,Topic 1,0.58
7,good etna adult child age feel spend money cab...,0.479371,0.520629,Topic 1,0.52
8,stay take half hour parking place choose go ea...,0.344991,0.655009,Topic 1,0.66
9,course must visit warn day even private tour v...,0.524117,0.475883,Topic 0,0.52


In [17]:
predictions_review = predictions_review.add_prefix('Review_')
predictions_review['Review_Dominant_Topic'] = predictions_review['Review_Dominant_Topic'].replace(' ', '_', regex=True)
predictions_review

,Review_content,Review_Topic_0,Review_Topic_1,Review_Dominant_Topic,Review_Perc_Dominant_Topic
0,arrive make sure allow plenty time mountain ro...,0.629743,0.370257,Topic_0,0.63
1,extraordinary active volcano beautiful landsca...,0.773659,0.226341,Topic_0,0.77
2,much cable car cost charge really look receipt...,0.479131,0.520869,Topic_1,0.52
3,small child august use company get see sight h...,0.202369,0.797631,Topic_1,0.80
4,able experience etna spend money tour research...,0.352371,0.647629,Topic_1,0.65
5,would visit old month old know take child youn...,0.443988,0.556012,Topic_1,0.56
6,take follow cable car star deduction bus trip ...,0.417466,0.582534,Topic_1,0.58
7,good etna adult child age feel spend money cab...,0.479371,0.520629,Topic_1,0.52
8,stay take half hour parking place choose go ea...,0.344991,0.655009,Topic_1,0.66
9,course must visit warn day even private tour v...,0.524117,0.475883,Topic_0,0.52


# 5. Merging data

In [18]:
data = pd.concat([predictions_title, predictions_review], axis=1)

data.head()

,Title_title,Title_Topic_0,Title_Topic_1,Title_Topic_2,Title_Dominant_Topic,Title_Perc_Dominant_Topic,Review_content,Review_Topic_0,Review_Topic_1,Review_Dominant_Topic,Review_Perc_Dominant_Topic
0,hard,0.166956,0.666083,0.166961,Topic_1,0.67,arrive make sure allow plenty time mountain ro...,0.629743,0.370257,Topic_0,0.63
1,must see active volcano least life,0.903465,0.048775,0.047760,Topic_0,0.90,extraordinary active volcano beautiful landsca...,0.773659,0.226341,Topic_0,0.77
2,cost cable car top,0.066697,0.066736,0.866568,Topic_2,0.87,much cable car cost charge really look receipt...,0.479131,0.520869,Topic_1,0.52
3,orgainise easy access,0.086296,0.083887,0.829817,Topic_2,0.83,small child august use company get see sight h...,0.202369,0.797631,Topic_1,0.80
4,cheap easy way,0.809144,0.083460,0.107396,Topic_0,0.81,able experience etna spend money tour research...,0.352371,0.647629,Topic_1,0.65


# 5. Classification model

In [19]:
predictions = predict_model(model_classification, data) 
predictions

,Title_title,Title_Topic_0,Title_Topic_1,Title_Topic_2,Title_Dominant_Topic,Title_Perc_Dominant_Topic,Review_content,Review_Topic_0,Review_Topic_1,Review_Dominant_Topic,Review_Perc_Dominant_Topic,Label,Score
0,hard,0.166956,0.666083,0.166961,Topic_1,0.67,arrive make sure allow plenty time mountain ro...,0.629743,0.370257,Topic_0,0.63,0,0.8292
1,must see active volcano least life,0.903465,0.048775,0.047760,Topic_0,0.90,extraordinary active volcano beautiful landsca...,0.773659,0.226341,Topic_0,0.77,0,0.9534
2,cost cable car top,0.066697,0.066736,0.866568,Topic_2,0.87,much cable car cost charge really look receipt...,0.479131,0.520869,Topic_1,0.52,1,0.6766
3,orgainise easy access,0.086296,0.083887,0.829817,Topic_2,0.83,small child august use company get see sight h...,0.202369,0.797631,Topic_1,0.80,1,0.6788
4,cheap easy way,0.809144,0.083460,0.107396,Topic_0,0.81,able experience etna spend money tour research...,0.352371,0.647629,Topic_1,0.65,0,0.9288
5,question cable car child,0.069453,0.067109,0.863438,Topic_2,0.86,would visit old month old know take child youn...,0.443988,0.556012,Topic_1,0.56,1,0.6770
6,cable car,0.111141,0.111143,0.777716,Topic_2,0.78,take follow cable car star deduction bus trip ...,0.417466,0.582534,Topic_1,0.58,1,0.6268
7,,0.333333,0.333333,0.333333,Topic_0,0.33,good etna adult child age feel spend money cab...,0.479371,0.520629,Topic_1,0.52,0,0.7612
8,,0.333333,0.333333,0.333333,Topic_0,0.33,stay take half hour parking place choose go ea...,0.344991,0.655009,Topic_1,0.66,0,0.7542
9,watch cloud,0.111266,0.774443,0.114291,Topic_1,0.77,course must visit warn day even private tour v...,0.524117,0.475883,Topic_0,0.52,0,0.8491
